In [ ]:

import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU') # 내 컴에 장착된 GPU를 list로 반환
try:
    tf.config.experimental.set_memory_growth(gpu[0], True) # GPU Memory Growth를 Enable
except RuntimeError as e:
    print(e) # Error 발생하면 Error 내용 출력

In [ ]:
from summarizer.domain.model.video import Video
import matplotlib

In [ ]:
video = Video(key="123",url="tests/2.mp4", algorithm="yolov3")
images = video._read_video()
idx, image = next(images)
print(image)

In [ ]:

features = image.extract()
print(features)
from summarizer.domain.model.feature import VideoFeature
videofeature = VideoFeature(key="123",features=features)

In [ ]:
video = Video(key="123",url="tests/2.mp4", algorithm="yolov3")
list_of_feature = video.extract_feature()

video.shorten(video_feature=list_of_feature, must_include_feature=["person"])

In [ ]:
import boto3


def create_movie_table(dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.create_table(
        TableName='Video',
        KeySchema=[
            {
                'AttributeName': 'key',
                'KeyType': 'HASH'  # Partition key
            },
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'key',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table


if __name__ == '__main__':
    movie_table = create_movie_table()
    print("Table status:", movie_table.table_status)

In [1]:
import boto3
from summarizer.infrastructure.repository import VideoRepository
dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")
videeoTable = dynamodb.Table('Video')
repo = VideoRepository(dynamodb, algorithm='yolov3')

In [5]:
videeoTable.scan()

{'Items': [{'end_time': None,
   'start_time': '2022/04/10 00:15:44',
   'key': '2022/04/10 00:15:44',
   'url': 'tests/2.mp4',
   'status': 'start'},
  {'end_time': None,
   'start_time': '2022/04/07 20:49:24',
   'key': '$2022/04/07 20:49:24',
   'url': 'tests/2.mp4',
   'status': 'start'},
  {'end_time': None,
   'start_time': '2022/04/07 20:51:10',
   'key': '$2022/04/07 20:51:10',
   'url': 'tests/2.mp4',
   'status': 'start'},
  {'end_time': None,
   'start_time': '2022/04/07 20:50:26',
   'key': '$2022/04/07 20:50:26',
   'url': 'tests/2.mp4',
   'status': 'start'},
  {'end_time': None,
   'start_time': '2022/04/07 20:52:39',
   'key': '$2022/04/07 20:52:39',
   'url': 'tests/2.mp4',
   'status': 'start'},
  {'end_time': None,
   'start_time': '2022/04/07, 20:46:55',
   'key': '$2022/04/07, 20:46:55',
   'url': 'tests/2.mp4',
   'status': 'start2'},
  {'end_time': None,
   'start_time': '2022/04/07 20:51:49',
   'key': '$2022/04/07 20:51:49',
   'url': 'tests/2.mp4',
   'status'

In [8]:
videeoTable.get_item(Key={"key":'2022/04/10 00:15:44'[Item]])

{'Item': {'end_time': None,
  'start_time': '2022/04/10 00:15:44',
  'key': '2022/04/10 00:15:44',
  'url': 'tests/2.mp4',
  'status': 'start'},
 'ResponseMetadata': {'RequestId': '39e625c0-d486-46ce-acde-779168462bda',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 09 Apr 2022 16:16:21 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '34169541',
   'x-amzn-requestid': '39e625c0-d486-46ce-acde-779168462bda',
   'content-length': '159',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [9]:
x = repo.get('2022/04/10 00:15:44')

ValidationError: 1 validation error for Video
key
  field required (type=value_error.missing)

In [ ]:
print(x)

In [ ]:
x = movie_table.get_item(Key={"key":'abc'})

In [ ]:
print(VideoFeature(**x['Item']))

In [ ]:
from summarizer.interface.settings import Settings
from summarizer.interface.sink_app import SinkApp


app = SinkApp(Settings())
app.run()

In [1]:
from summarizer.infrastructure.redis_listener import RedisListener
from redis import Redis
import json
redis = Redis(host="127.0.0.1", port="6379")
from summarizer.service.command import ExtractFeature
string = json.dumps(ExtractFeature(type="ExtractFeature", key='2022/04/10 00:15:44').dict())
print(string,type(string))
redis.lpush('summarizer',string)
# x = redis.rpop('summarizer')
# x = json.loads(x)
# print(type(x), x)


TypeError: Object of type ExtractFeature is not JSON serializable

In [ ]:
from summarizer.interface.container import Container